In [79]:
import pandas as pd
from pandas.testing import assert_frame_equal
import numpy as np
import csv 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split

In [80]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', 5000)

In [81]:
# Read csv
asthma_df = pd.read_csv('astma.csv')
healthy_df = pd.read_csv("healthy_parsed.csv")

In [82]:
# Get the name of all the columns
print(healthy_df.columns)
print(asthma_df.columns)

Index(['Index', 'SubjectNr', 'DayNo', 'weekday', 'dayType', 'Age', 'sex', 'weight', 'height', 'BMI_SDS',
       ...
       'FHX', 'FHN', 'TG', 'TN', 'TX', 'SQ', 'SP', 'DR', 'RH', 'RHX'], dtype='object', length=111)
Index(['Unnamed: 0', 'SubjectNr', 'DayNo', 'weekday', 'dayType', 'stepsTotalDaily', 'stepsTotalDetailed', 'steps00', 'steps01', 'steps02',
       ...
       'wear24H', 'wear16H', 'steps15_19', 'week', 'steps_hour_max', 'AVGHR_daily', 'AVGHR_wake', 'AVGHR_sleep', 'CONDITION', 'EventDay'], dtype='object', length=214)


# Data analysis

In [83]:
healthy_df.head(5)

,Index,SubjectNr,DayNo,weekday,dayType,Age,sex,weight,height,BMI_SDS,ethnicity,school_year_final,sportsyesno,urbanisation,PedsQL_score_baseline,stepsTotalDaily,steps_hour_max,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HRMinSleep,HRMaxSleep,AVGHR_daily,AVGHR_sleep,AVGHR_wake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,wear05H,wear16H,wear24H,BODY_TEMPERATURE_DEG_C,DIASTOLIC_BLOOD_PRESSURE_MMHG,HEART_PULSE_BPM,SYSTOLIC_BLOOD_PRESSURE_MMHG,WEIGHT_KG,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,sleeptime,waketime,fvc_best,fev1_best,pef_best,grade_fev1,grade_fvc,predicted_fvc_best,predicted_fev1_best,predicted_fev1_ratio_best,fev1_ratio_best,fev1_percentage,fvc1_percentage,fev1_ratio_percentage,school_yes_no,screentime,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX
0,1,2853189,0,Thu,holiday,2,Female,12.0,92.0,-1.8,Other / Mixed,Day care,No,Extremely urbanised,97.826087,2806.0,1992.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,480.0,1992.0,156.0,51.0,61.0,39.0,NaN,20.0,56.0,123.0,NaN,NaN,96.0,NaN,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.5,106.5,109.000000,NaN,56.0,NaN,92.0,0,44,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.6,8,4,5.2,3.4,7.5,2.0,25,1.9,9,2
1,2,2853189,1,Fri,holiday,2,Female,12.0,92.0,-1.8,Other / Mixed,Day care,No,Extremely urbanised,97.826087,2564.0,1205.0,151.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,9.0,58.0,NaN,NaN,NaN,1205.0,599.0,NaN,NaN,NaN,52.0,163.0,NaN,NaN,126.0,119.0,128.0,119.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.500000,113.6,145.0,NaN,NaN,20,38,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neither,NaN,7.9,9,4,5.9,4.4,7.2,0.0,0,0.0,-1,-1
2,3,2853189,2,Sat,holiday,2,Female,12.0,92.0,-1.8,Other / Mixed,Day care,No,Extremely urbanised,97.826087,521.0,369.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,369.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN,74.0,NaN,NaN,NaN,NaN,71.0,137.0,NaN,NaN,103.0,NaN,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,NaN,NaN,NaN,NaN,NaN,NaN,102.5,100.333333,NaN,NaN,NaN,NaN,0,25,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neither,NaN,9.2,11,8,7.1,6.0,7.8,0.0,0,0.2,1,1
3,4,2853189,3,Sun,holiday,2,Female,12.0,92.0,-1.8,Other / Mixed,Day care,No,Extremely urbanised,97.826087,250.0,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.0,106.0,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,151.0,156.0,NaN,NaN,154.0,NaN,154.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,19,12,38.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3,8,1,6.4,4.5,7.0,0.0,0,0.0,-1,-1
4,5,2853189,4,Mon,school,2,Female,12.0,92.0,-1.8,Other / Mixed,Day care,No,Extremely urbanised,97.826087,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,6,4,38.157143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neither,NaN,7.3,14,1,8.0,4.7,9.8,0.0,0,0.6,3,1


In [84]:
asthma_df.head(5)

,Unnamed: 0,SubjectNr,DayNo,weekday,dayType,stepsTotalDaily,stepsTotalDetailed,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HrAvgWake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,HRAvgSleep,HRMinSleep,HRMaxSleep,activity_score_parent,school_yes_no,activity_score_child,screentime,sleep_score_child,bedtimeReport,waketimeReport,fvc_best,fev1_best,pef_best,grade_fev1,grade_fvc,Technique,fev1_ratio_best,fev1_percentage,fvc1_percentage,fev1_ratio_percentage,ACD1,ACD2,ACD3,ACD4,ACD5,ACD6,ACD6_use,ACD6score,BASELINE_ACQ6_score,EOS_ACQ6_score,BASELINE_physical_tot,EOS_physical_tot,BASELINE_emotional_tot,EOS_emotional_tot,BASELINE_school_tot,EOS_school_tot,BASELINE_bodyimage_tot,EOS_bodyimage_tot,BASELINE_treatmentburden_tot,EOS_treatmentburden_tot,BASELINE_health_tot,EOS_health_tot,BASELINE_weight_tot,EOS_weight_tot,BASELINE_respiratory_tot,EOS_respiratory_tot,BASELINE_digestion_tot,EOS_digestion_tot,BASELINE_PAQLQ_total,EOS_PAQLQ_total,BASELINE_PAQLQ_activity,EOS_PAQLQ_activity,BASELINE_PAQLQ_symptoms,EOS_PAQLQ_symptoms,BASELINE_PAQLQ_emotion,EOS_PAQLQ_emotion,BASELINE_PedsQL_score,EOS_PedsQL_score,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX,pollenTotal,NO,NO2,Nox,O3,Ox,PM10,PM25,Symptomscore,Admission_LY,Age,AgeDiagnosis,AllergicRhin,AtopicAsthma,BirthMonth,BirthYear,BMI,BMI_SDS,CF_Asthma,DailyActiv,DayCare,Dis_Chronic,Eosinophils,ExerciseSymp,Family_his,Gender,Height,Hosp_treatme,Hospital_Trt,ICS,LABA,Leukotri_Mod,Monoclonals,Oral_Steroid,OtherClass,PancrInsuffi,PetsHome,Pseudomonas,SABA,SchoolYear,Serum_IgE,SmokeHome,Sports,Vaccination,Weight,school_year_edit,Urbanisation,urbanisation2,totalSleepDuration,wear00,wear01,wear02,wear03,wear04,wear05,wear06,wear07,wear08,wear09,wear10,wear11,wear12,wear13,wear14,wear15,wear16,wear17,wear18,wear19,wear20,wear21,wear22,wear23,wear24H,wear16H,steps15_19,week,steps_hour_max,AVGHR_daily,AVGHR_wake,AVGHR_sleep,CONDITION,EventDay
0,1,1771258,0,Tue,school,3723.0,3723.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,137.0,NaN,NaN,605.0,1640.0,NaN,82.0,451.0,245.0,135.0,160.0,164.0,41.0,NaN,68.0,120.0,8.689362e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.200000e+02,NaN,NaN,NaN,NaN,NaN,8.985714e+09,8.342857e+09,8.787500e+04,9.150000e+01,9.337500e+04,7.716667e+09,7.866667e+09,660.0,16800.0,16440.0,1.0,75.0,61.0,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3232.0,2381.0,4334.00,D,B,3.0,0.736696,0.726555,0.877857,0.822316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,2.666667e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.652174e+09,5.304348e+09,6.4,5.6,6.5,4.2,7.0,6.5,847826087.0,8.586957e+09,6.3,8,4,14.0,85,19.7,5.4,58,0.0,0,0,6.0,32.31,48.88,98.41,3.86,27.83,23.42,13.04,NaN,1,15,7.0,No,Yes,12,2002.0,19.5,-0.1,Controlled asthma,School,0.0,0.0,NaN,No,No,Female,163.2,0.0,0.0,Yes,Yes,No,No,No,No,NaN,Yes,NaN,Yes,4.0,1465.0,No,Yes,0.0,51.9,Secondary school year 4,1.0,Extremely urbanised,33240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,54.0,69.0,NaN,1.0,1640.0,90.0,94.0,NaN,Controlled asthma,NaN
1,2,1771258,1,Wed,school,10015.0,10015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,607.0,580.0,325.0,180.0,322.0,181.0,491.0,161.0,10.0,744.0,1146.0,4355.0,722.0,142.0,49.0,NaN,82.0,168.0,1.326242e+09,7.360000e+01,7.528571e+09,7.075000e+01,9.200000e+01,7.600000e+01,7.033333e+09,8.233333e+09,78.0,9.900000e+01,1.130000e+02,1.298000e+02,9.516667e+09,9.166667e+09,NaN,108.5,8.525000e+01,9.340000e+01,9.588889e+09,1.018333e+09,1.423074e+09,1.078154e+09,1.042000e+02,9.183333e+09,9.300000e+01,180.0,22200.0,11760.0,0.0,84.0,67.0,121.0,8.0,Ja,8.0,B. 30 tot 60 minuten,9.0,15:34:00,13:04:00,3261.0,2257.0,4509.00,A,A,3.0,0.692119,0.688716,0.885734,0.772559,0.0,0.0,1.0,2.0,0.0,0.0,

# Preprocessing

## Removing outliers

## Removing columns with too much missing data

In this section, we will remove the columns having too many missing values. First for the healthy kids, then for the sick ones.

### Healthy kids

In [85]:
# Compute for all the columns, the percentages of Nan per column
# We will drop the columns that have more than 50% Nan
percentNan = []
for col in healthy_df.columns:
    nan_percentage = healthy_df[col].isnull().sum() / len(healthy_df[col]) * 100
    percentNan.append((nan_percentage, col))

percentNan.sort(reverse=True)

for el in percentNan:
    if el[0] > 50:
        print(el)


(99.63636363636364, 'fev1_ratio_best')
(98.46753246753246, 'steps04')
(98.28571428571429, 'steps03')
(97.42857142857143, 'steps02')
(96.44155844155844, 'steps05')
(95.8961038961039, 'steps01')
(91.76623376623377, 'steps00')
(87.81818181818181, 'WEIGHT_KG')
(81.4025974025974, 'steps23')
(80.9090909090909, 'grade_fvc')
(80.9090909090909, 'grade_fev1')
(80.5974025974026, 'predicted_fvc_best')
(80.5974025974026, 'predicted_fev1_ratio_best')
(80.5974025974026, 'predicted_fev1_best')
(80.5974025974026, 'pef_best')
(80.5974025974026, 'fvc_best')
(80.5974025974026, 'fvc1_percentage')
(80.5974025974026, 'fev1_ratio_percentage')
(80.5974025974026, 'fev1_percentage')
(80.5974025974026, 'fev1_best')
(71.94805194805195, 'BODY_TEMPERATURE_DEG_C')
(69.32467532467533, 'steps06')
(66.23376623376623, 'steps22')


Looking at the percentages, we can delete (for the healthy kids) the following data (if we keep only the column with less than 50% of missing data):
. fev1_ratio_best
. grade_fvc
- grade_fev1
- predicted_fvc_best
- predicted_fev1_ratio_best
- predicted_fev1_best
- pef_best
- fvc_best
- fvc1_percentage
- fev1_ratio_percentage
- fev1_percentage
- fev1_best
- BODY_TEMPERATURE_DEG_C


In [86]:
#Remove the desired columns
# healthy_df = healthy_df.drop(columns=["fev1_ratio_best", "grade_fvc", "grade_fev1", "predicted_fvc_best", "predicted_fev1_ratio_best", "predicted_fev1_best", "pef_best", "fvc_best", "fvc1_percentage", "fev1_ratio_percentage", "fev1_percentage", "fev1_best", "BODY_TEMPERATURE_DEG_C"])

### Sick kids

In [87]:
# Compute for all the columns, the percentages of Nan per column
# We will drop the columns that have more than 50% Nan
percentNan = []
for col in asthma_df.columns:
    nan_percentage = asthma_df[col].isnull().sum() / len(asthma_df[col]) * 100
    percentNan.append((nan_percentage, col))

percentNan.sort(reverse=True)

for el in percentNan:
    if el[0] > 50:
        print(el)

(98.35375191424195, 'steps03')
(98.31546707503828, 'steps04')
(97.70290964777948, 'steps02')
(96.40122511485451, 'steps05')
(96.32465543644717, 'steps01')
(92.26646248085758, 'EventDay')
(90.5436447166922, 'steps00')
(79.28790199081163, 'steps23')
(77.90964777947933, 'steps06')
(76.64624808575805, 'EOS_weight_tot')
(76.64624808575805, 'EOS_treatmentburden_tot')
(76.64624808575805, 'EOS_school_tot')
(76.64624808575805, 'EOS_respiratory_tot')
(76.64624808575805, 'EOS_physical_tot')
(76.64624808575805, 'EOS_health_tot')
(76.64624808575805, 'EOS_emotional_tot')
(76.64624808575805, 'EOS_digestion_tot')
(76.64624808575805, 'EOS_bodyimage_tot')
(75.57427258805512, 'Eosinophils')
(74.23430321592649, 'Symptomscore')
(70.02297090352221, 'Serum_IgE')
(68.60643185298622, 'ACD6')
(67.72588055130169, 'Pseudomonas')
(67.72588055130169, 'BASELINE_weight_tot')
(67.72588055130169, 'BASELINE_treatmentburden_tot')
(67.72588055130169, 'BASELINE_school_tot')
(67.72588055130169, 'BASELINE_respiratory_tot')
(

Looking at the percentages, we can delete (for the healthy kids) the following data (if we keep only the column with less than 50% of missing data):

- Eosinophils
- sleep_score_child
- screentime
- school_yes_no
- bedtimeReport
- activity_score_parent
- activity_score_child

Need to look more into: * Serum_IgE, ACD1, ACD2, ACD3, ACD4, ACD5, ACD6, Eosinophils *

In [88]:
#Remove the columns mentioned above
# asthma_df = asthma_df.drop(columns=["Eosinophils", "sleep_score_child", "screentime", "school_yes_no", "bedtimeReport", "activity_score_parent", "activity_score_child"])

#### Technique
About technique 2 Matthijs wrote this:

Is it possible to do your analyses with both options to see whether it impacts the performance of your models? If not, I would just use the ones marked ‘2’ anyway to include as much data as possible.


In [89]:
asthma_df['Technique'].value_counts()
asthma_df = asthma_df.drop(asthma_df[asthma_df['Technique'] == 1].index).reset_index().drop('index', axis=1)
# IF WE DELETE THE WRONG TECHNIQUE HERE, ANY dadaset.loc[i] WILL GIVE AN ERROR AT THE FIRST MISSING ROW
# We need to reset the indicies to avoid this

## Removing rows with too many missing values

In [90]:
# # TODO

# # Subject 2429672 has no data about gender, sports and urbanisation 
# asthma_df = asthma_df[asthma_df['SubjectNr'] != 2429672]

# Alligning column names

In [91]:
# Renaming columns (allignment)
asthma_df = asthma_df.rename(columns={'Gender': 'sex'})
asthma_df = asthma_df.rename(columns={'Weight': 'weight'})
asthma_df = asthma_df.rename(columns={'Height': 'height'})

# What grade they are in
healthy_df = healthy_df.rename(columns={'school_year_final': 'school_year'})
asthma_df = asthma_df.rename(columns={'school_year_edit': 'school_year'})

# Wheter they do a sport
asthma_df = asthma_df.rename(columns={'Sports': 'sportsyesno'})

# Note there are 2 urbanisations
asthma_df = asthma_df.rename(columns={'urbanisation2': 'urbanisation'})

# pedsql
asthma_df = asthma_df.rename(columns={'BASELINE_PedsQL_score': 'PedsQL_score_baseline'})

# Note healthy_df doesn't have all wear variables (it only has the grped vars)
# Asthma_df doesnt have wear05H lets compute it based on the other wear vars
asthma_df['wear05H'] = (asthma_df.loc[:, 'wear00':'wear05'] == 1).mean(axis=1) * 100

# No temperature in asthma_df but is in healthy_df?
# Drop the col
healthy_df = healthy_df.drop('BODY_TEMPERATURE_DEG_C', axis=1)

# No blood preassure in asthma_df but is in healthy_df?
# drop in healthy
healthy_df = healthy_df.drop('DIASTOLIC_BLOOD_PRESSURE_MMHG', axis=1)
healthy_df = healthy_df.drop('SYSTOLIC_BLOOD_PRESSURE_MMHG', axis=1)

# Note that WEIGHT_KG is a daily meassurement in healthy_df
# in asthma_df this meassurement is only taken at begin and end of the study period
# Also missing data is 87% lets drop this col
healthy_df = healthy_df.drop('WEIGHT_KG', axis=1)

# I am assuming these columns are the same (bedtimeReport and waketimeReport not in legend))
asthma_df = asthma_df.rename(columns={'bedtimeReport': 'sleeptime'})
asthma_df = asthma_df.rename(columns={'waketimeReport': 'waketime'})

# Create a new column with the hourly categories starting from 0 to 23
hour_mapping = {i: i for i in range(24)}

# Converting to datetime
healthy_df['sleeptime'] = pd.to_datetime(healthy_df['sleeptime'])
asthma_df['sleeptime'] = pd.to_datetime(asthma_df['sleeptime'])
healthy_df['waketime'] = pd.to_datetime(healthy_df['waketime'])
asthma_df['waketime'] = pd.to_datetime(asthma_df['waketime'])

# Setting hourly value
healthy_df['sleeptime'] = healthy_df['sleeptime'].dt.hour
asthma_df['sleeptime'] = asthma_df['sleeptime'].dt.hour
healthy_df['waketime'] = healthy_df['waketime'].dt.hour
asthma_df['waketime'] = asthma_df['waketime'].dt.hour

# predicted_fvc_best, predicted_fev1_best, predicted_fev1_ratio_best
# All of these not in asthma
healthy_df = healthy_df.drop('predicted_fvc_best', axis=1)
healthy_df = healthy_df.drop('predicted_fev1_best', axis=1)
healthy_df = healthy_df.drop('predicted_fev1_ratio_best', axis=1)

# Screentime, values need to be alligned 
# Alignment dictionary
alignment_dict = {
    '0': '0',
    'D. 2 uur tot 4 uur': '2-4 hours',
    'A. 0 tot 30 minuten': '0-30 min',
    'C. 1 uur tot 2 uur': '1-2 hours',
    'B. 30 tot 60 minuten': '0.5-1 hours',
    'E. Meer dan 4 uur': '> 4 hours'
}

# Rename values in the first dataframe based on the alignment dictionary
asthma_df['screentime'] = asthma_df['screentime'].replace(alignment_dict)

# Dealing with missing data


## Issue with HR columns

In the asthma dataset there seems to be a problem with HR00 to HR23. Sometimes, the heart rate goes above a million sometimes. It seems like the first couple digits correspond with the expected heart rate.

In [92]:
# In the asthma dataset there seems to be a problem with HR00 to HR23
# The heart rate goes above a million sometimes
# It seems like the first couple digits correspond with the expected heart rate.
# 2 cases: HR below 100 and HR above 100

# Loop through each column in the dataframe
for col in asthma_df.loc[:, "HR00":"HR23"]:
    for i in range(len(asthma_df[col])):
        if (asthma_df.loc[i, col] > 1000):
            # Take first 2 digits of float
            first_2_digits = str(asthma_df.loc[i, col])[:2]
            temp_number = float(first_2_digits)
            # If heart rate lower than 20 it means it should be greater than 100 (assuming heart rates < 200)
            if temp_number < 20:
                val = str(asthma_df.loc[i, col])[:3] + '.' + str(asthma_df.loc[i, col])[3]
                asthma_df.loc[i, col] = float(val)
            else:
                val = str(asthma_df.loc[i, col])[:2] + '.' + str(asthma_df.loc[i, col])[2]
                asthma_df.loc[i, col] = float(val)
            pass

## Child exrtaction

Let's first separate the databese into chilferen. This will create on database per children in order to identify them more easily. This can be useful if one data is repeated only once per child. We could then spread the data through the child.£

In [93]:
# Create a new database per children and save it into a list of all the same subject numbers.
healthy_children = []
asthma_children = []

# Loop through all the subject numbers
for subject in healthy_df['SubjectNr'].unique():
    healthy_children.append(healthy_df[healthy_df['SubjectNr'] == subject])

for subject in asthma_df['SubjectNr'].unique():
    asthma_children.append(asthma_df[asthma_df['SubjectNr'] == subject])

## Dealing with Nan's values.

### StepsXX
this still leaves Nans in the datasets. We have to work on steps AFTER HR filling (we can't put everything after HR, bc we would get lots of 0s during daytime where we had no HR, but this doesn't mean they weren't moving). Maybe do the same what we did on HR filling/guessing. But BOTH needs to run on the datasets.
- First changing the NaNs to 0s where we have HR, bc it means they weren't moving
- Second filling the remaining NaNs by the AVGs
    - somewhere it will give some numbers (> 0) to night hours, bc the AVGs. If it's a problem, we could run the previous solution on these night-hours
    
------
With steps I think we should focus steps09-steps20, because before it and after it the number of records are dropping fast (probably bc they're sleeping) -> steps21-steps08 Nan could be 0 **CAN USE waketimeReport **

should check the % of the missing data of the remaining stepsXXs

In [94]:
# TODO: What is this one doing?
def fill_stepsXX(db):
    for i in range(len(db)):
        for hour in ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']:
            if db.iloc[i][f'HR{hour}'] >= 0 and np.isnan(db.iloc[i][f'steps{hour}']):
                db.loc[i, f'steps{hour}'] = 0


# fill_stepsXX(asthma_df)
# fill_stepsXX(healthy_df)

In [95]:
healthy_subject_nrs = healthy_df['SubjectNr'].drop_duplicates()
asthma_subject_nrs = asthma_df['SubjectNr'].drop_duplicates()

In [96]:
healthy_df.loc[:, 'steps00':'steps08'] = healthy_df.loc[:, 'steps00':'steps08'].fillna(0)
healthy_df.loc[:, 'steps21':'steps23'] = healthy_df.loc[:, 'steps21':'steps23'].fillna(0)
asthma_df.loc[:, 'steps00':'steps08'] = asthma_df.loc[:, 'steps00':'steps08'].fillna(0)
asthma_df.loc[:, 'steps21':'steps23'] = asthma_df.loc[:, 'steps21':'steps23'].fillna(0)

In [97]:
Mon_steps_AVGs = pd.DataFrame()
Tue_steps_AVGs = pd.DataFrame()
Wed_steps_AVGs = pd.DataFrame()
Thu_steps_AVGs = pd.DataFrame()
Fri_steps_AVGs = pd.DataFrame()
Sat_steps_AVGs = pd.DataFrame()
Sun_steps_AVGs = pd.DataFrame()
# no two SubjectNr are the same, so we don't need separated dataframes for asthma and healthy

def fill_avg_steps(database, subjects):
    for subject in subjects:
        Mon_steps_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Mon'), 'steps00':'steps23'].describe().loc['mean'],0)
        Tue_steps_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Tue'), 'steps00':'steps23'].describe().loc['mean'],0)
        Wed_steps_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Wed'), 'steps00':'steps23'].describe().loc['mean'],0)
        Thu_steps_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Thu'), 'steps00':'steps23'].describe().loc['mean'],0)
        Fri_steps_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Fri'), 'steps00':'steps23'].describe().loc['mean'],0)
        Sat_steps_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Sat'), 'steps00':'steps23'].describe().loc['mean'],0)
        Sun_steps_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Sun'), 'steps00':'steps23'].describe().loc['mean'],0)

fill_avg_steps(asthma_df, asthma_subject_nrs)
fill_avg_steps(healthy_df, healthy_subject_nrs)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13116\1943150399.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Mon_steps_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Mon'), 'steps00':'steps23'].describe().loc['mean'],0)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13116\1943150399.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Tue_steps_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Tue'), 'steps00':'steps23'].desc

In [98]:
Mon_steps_AVGs = Mon_steps_AVGs.transpose()
Tue_steps_AVGs = Tue_steps_AVGs.transpose()
Wed_steps_AVGs = Wed_steps_AVGs.transpose()
Thu_steps_AVGs = Thu_steps_AVGs.transpose()
Fri_steps_AVGs = Fri_steps_AVGs.transpose()
Sat_steps_AVGs = Sat_steps_AVGs.transpose()
Sun_steps_AVGs = Sun_steps_AVGs.transpose()

In [99]:
def fill_stepsxx(db):
    for stepsxx in db.loc[:, 'steps00':'steps23'].columns.to_list():
        db[stepsxx] = db.apply(
            lambda row: Mon_steps_AVGs.loc[row['SubjectNr']][stepsxx] if np.isnan(row[stepsxx]) and row['weekday'] == 'Mon' else 
                        Tue_steps_AVGs.loc[row['SubjectNr']][stepsxx] if np.isnan(row[stepsxx]) and row['weekday'] == 'Tue' else
                        Wed_steps_AVGs.loc[row['SubjectNr']][stepsxx] if np.isnan(row[stepsxx]) and row['weekday'] == 'Wed' else
                        Thu_steps_AVGs.loc[row['SubjectNr']][stepsxx] if np.isnan(row[stepsxx]) and row['weekday'] == 'Thu' else
                        Fri_steps_AVGs.loc[row['SubjectNr']][stepsxx] if np.isnan(row[stepsxx]) and row['weekday'] == 'Fri' else
                        Sat_steps_AVGs.loc[row['SubjectNr']][stepsxx] if np.isnan(row[stepsxx]) and row['weekday'] == 'Sat' else
                        Sun_steps_AVGs.loc[row['SubjectNr']][stepsxx] if np.isnan(row[stepsxx]) and row['weekday'] == 'Sun' else
                        row[stepsxx],
            axis=1
        )


fill_stepsxx(asthma_df)
fill_stepsxx(healthy_df)

In [100]:
healthy_df.loc[:, 'steps00':'steps23'].isna().sum()

steps00     0
steps01     0
steps02     0
steps03     0
steps04     0
steps05     0
steps06     0
steps07     0
steps08     0
steps09    59
steps10    60
steps11    62
steps12    69
steps13    76
steps14    81
steps15    56
steps16    58
steps17    64
steps18    59
steps19    58
steps20    62
steps21     0
steps22     0
steps23     0
dtype: int64

### stepsTotalDaily
If we guess the missing steps, we can count it (we have 0 here, if ALL the steps are missing)
comment: How do you want to guess the missing steps?
Ákos's comment: Yeaah, I meant that IF we can (want to) guess the steps, this variable won't be a problem haha. But I couldn't figure out any good method to do it. Maybe the avg steps of the same hour on different weeks. Like they have P.E. on Tuesday 9-11, so they'll have more steps this time of the week.

In [101]:
def fill_stdaily(db):
    for i in range(len(db)):
        db.loc[i, 'stepsTotalDaily'] = db.loc[i, 'steps00':'steps23'].sum()

fill_stdaily(asthma_df)
fill_stdaily(healthy_df)

### weight & height
If we know the age, we could add the avg weight and height to a kid. The average height and weight are extracted from the following website: www.disabled-world.com/calculators-charts/height-weight-teens.php

Sick kids do not have weight, but they have their BMI. On the other hand, healthy kids do not have BMI but they have their weight.

In [102]:
healthy_df[['weight', 'height', 'BMI_SDS']].isna().sum()

weight     286
height      44
BMI_SDS    286
dtype: int64

In [103]:
asthma_df[['weight', 'height', 'BMI_SDS']].isna().sum()

weight     29
height     29
BMI_SDS     0
dtype: int64

In [104]:
f_awh = pd.DataFrame(np.array([
    [2, 12.02, 85.5],
    [3, 14.29, 94],
    [4, 15.42, 100.3],
    [5, 17.92, 107.9],
    [6, 19.96, 115.5],
    [7, 22.45, 121.1],
    [8, 25.85, 128.2],
    [9, 28.12, 133.3],
    [10, 31.98, 138.4],
    [11, 36.97, 144.0],
    [12, 41.5, 149.8],
    [13, 45.81, 156.7],
    [14, 47.63, 158.7],
    [15, 52.16, 159.7],
    [16, 53.52, 162.5]
]),
columns=['Age', 'weight', 'height'])

m_awh = pd.DataFrame(np.array([
    [2, 12.47, 86.8],
    [3, 14.06, 95.2],
    [4, 16.33, 102.3],
    [5, 18.37, 109.2],
    [6, 20.64, 115.5],
    [7, 22.9, 121.9],
    [8, 25.63, 128.0],
    [9, 28.58, 133.3],
    [10, 32.0, 138.4],
    [11, 35.6, 143.5],
    [12, 39.92, 149.1],
    [13, 45.36, 156.2],
    [14, 50.8, 163.8],
    [15, 56.02, 170.1],
    [16, 60.78, 173.4]
]),
columns=['Age', 'weight', 'height'])

In [105]:
def fill_w_h(db, ref, column, sex_Female):
    db[column] = db.apply(
        lambda row: ref.loc[ref['Age'] == row['Age'], [column]].iloc[0][column] if np.isnan(row[column]) and row['sex'] == sex_Female else row[column],
        axis=1
    )

In [106]:
fill_w_h(healthy_df, f_awh, 'weight', "Female")
fill_w_h(healthy_df, f_awh, 'height', "Female")
fill_w_h(healthy_df, m_awh, 'weight', "Male")
fill_w_h(healthy_df, m_awh, 'height', "Male")
# We don't need for asthma_df, since we have no missing data there

In [107]:
asthma_df[['weight', 'height', 'BMI_SDS']].isna().sum()

# We have 29 data we couldn't guess, bc we don't know the child's sex. Should we guess?
# The weight/height values are almost the same at this age (12)

weight     29
height     29
BMI_SDS     0
dtype: int64

### BMI_SDS
It's missing only bc the weight or the weight+height is missing.

In [108]:
# We don't need to do this for asthma_df

healthy_df['BMI'] = healthy_df['weight'] / (healthy_df['height']*healthy_df['height']/10000)

for age in range(2,17):
    mean = healthy_df.loc[healthy_df['Age'] == age, ['BMI']].describe().loc['mean'][0]
    std = healthy_df.loc[healthy_df['Age'] == age, ['BMI']].describe().loc['std'][0]
    healthy_df.loc[healthy_df['Age'] == age, ['BMI_SDS']] = np.round((healthy_df['BMI'] - mean) / std, 1)

healthy_df = healthy_df.drop(['BMI'], axis=1)

In [109]:
healthy_df[['weight', 'height', 'BMI_SDS']].isna().sum()

weight     0
height     0
BMI_SDS    0
dtype: int64

In [110]:
asthma_df[['weight', 'height', 'BMI_SDS']].isna().sum()

# We have 29 data we couldn't guess, bc we don't know the child's sex. Should we guess?
# The weight/height values are almost the same at this age (12)

weight     29
height     29
BMI_SDS     0
dtype: int64

In [111]:
asthma_df.loc[np.isnan(asthma_df['weight']), ['Age','weight','height','sex']]

,Age,weight,height,sex
1467,12,NaN,NaN,NaN
1468,12,NaN,NaN,NaN
1469,12,NaN,NaN,NaN
1470,12,NaN,NaN,NaN
1471,12,NaN,NaN,NaN
1472,12,NaN,NaN,NaN
1473,12,NaN,NaN,NaN
1474,12,NaN,NaN,NaN
1475,12,NaN,NaN,NaN
1476,12,NaN,NaN,NaN


### HR00-HR23
Daniel has good ideas about these. Avg Tuesday(e.g.), or prev. and next day, or smth else.
Take a kid and replace the Nan values in the HR using the other similar days of the same kid.

In [112]:
Mon_HR_AVGs = pd.DataFrame()
Tue_HR_AVGs = pd.DataFrame()
Wed_HR_AVGs = pd.DataFrame()
Thu_HR_AVGs = pd.DataFrame()
Fri_HR_AVGs = pd.DataFrame()
Sat_HR_AVGs = pd.DataFrame()
Sun_HR_AVGs = pd.DataFrame()
# no two SubjectNr are the same, so we don't need separated dataframes for asthma and healthy

def fill_avg_hr(database, subjects):
    for subject in subjects:
        Mon_HR_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Mon'), 'HR00':'HR23'].describe().loc['mean'],2)
        Tue_HR_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Tue'), 'HR00':'HR23'].describe().loc['mean'],2)
        Wed_HR_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Wed'), 'HR00':'HR23'].describe().loc['mean'],2)
        Thu_HR_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Thu'), 'HR00':'HR23'].describe().loc['mean'],2)
        Fri_HR_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Fri'), 'HR00':'HR23'].describe().loc['mean'],2)
        Sat_HR_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Sat'), 'HR00':'HR23'].describe().loc['mean'],2)
        Sun_HR_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Sun'), 'HR00':'HR23'].describe().loc['mean'],2)

fill_avg_hr(asthma_df, asthma_subject_nrs)
fill_avg_hr(healthy_df, healthy_subject_nrs)

Mon_HR_AVGs.isna().sum() # Some subject has 24 NaNs = they have 0 record on Mondays

C:\Users\Admin\AppData\Local\Temp\ipykernel_13116\3331437980.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Mon_HR_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Mon'), 'HR00':'HR23'].describe().loc['mean'],2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13116\3331437980.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Tue_HR_AVGs[subject] = np.round(database.loc[(database['SubjectNr'] == subject) & (database['weekday'] == 'Tue'), 'HR00':'HR23'].describe().loc['mean']

1771258     0
1125968     0
1137963     0
8380204     0
1725729     0
4016957     0
8251413     0
4484964     0
8438039     0
3864661     0
2404542     0
4957084     1
2461821     0
1353701     6
5557553     0
7612162     0
5682271     0
1086145     0
8804324     0
3167719     0
1282713     0
4877940     0
8150231     0
4651197     0
5302478     0
2985100     0
4571581     0
1642849     0
1438514     0
2450908     8
7190055     0
3175094     0
8628998     0
2102215     0
5995703     0
4962375     0
8364553     0
2689839     0
1921897     0
4581506     0
5637993     0
2492797     0
1334205     0
4936000     0
3085968     0
4286442     0
5461205     1
2665460     0
3495507     0
8488946     0
5158754     0
2429672     0
8466277     0
6115426     0
5310691     0
4795179     0
3004792     0
1130656     0
8125067     0
5578891     7
4722486     0
8533888     0
8635768     1
8196400     0
5039256     0
4986804     0
4745136     0
1925533     0
3132734     0
3831912     0
3762951     0
735834

In [113]:
Mon_HR_AVGs = Mon_HR_AVGs.transpose()
Tue_HR_AVGs = Tue_HR_AVGs.transpose()
Wed_HR_AVGs = Wed_HR_AVGs.transpose()
Thu_HR_AVGs = Thu_HR_AVGs.transpose()
Fri_HR_AVGs = Fri_HR_AVGs.transpose()
Sat_HR_AVGs = Sat_HR_AVGs.transpose()
Sun_HR_AVGs = Sun_HR_AVGs.transpose()

In [114]:
def fill_hrxx(db):
    for hrxx in db.loc[:, 'HR00':'HR23'].columns.to_list():
        db[hrxx] = db.apply(
            lambda row: Mon_HR_AVGs.loc[row['SubjectNr']][hrxx] if np.isnan(row[hrxx]) and row['weekday'] == 'Mon' else 
                        Tue_HR_AVGs.loc[row['SubjectNr']][hrxx] if np.isnan(row[hrxx]) and row['weekday'] == 'Tue' else
                        Wed_HR_AVGs.loc[row['SubjectNr']][hrxx] if np.isnan(row[hrxx]) and row['weekday'] == 'Wed' else
                        Thu_HR_AVGs.loc[row['SubjectNr']][hrxx] if np.isnan(row[hrxx]) and row['weekday'] == 'Thu' else
                        Fri_HR_AVGs.loc[row['SubjectNr']][hrxx] if np.isnan(row[hrxx]) and row['weekday'] == 'Fri' else
                        Sat_HR_AVGs.loc[row['SubjectNr']][hrxx] if np.isnan(row[hrxx]) and row['weekday'] == 'Sat' else
                        Sun_HR_AVGs.loc[row['SubjectNr']][hrxx] if np.isnan(row[hrxx]) and row['weekday'] == 'Sun' else
                        row[hrxx],
            axis=1
        )


fill_hrxx(asthma_df)
fill_hrxx(healthy_df)

### HR05Perc and HR95Perc
If we guess the missing HRs, we can count it (we have NaN here, if ALL the HRs are missing)

Talk with Rachel about those lines. The values already in the database do not correspond.

In [115]:
# TODO: act upon Teacher's response on the above question.

# asthma_df.loc[asthma_df['SubjectNr'] == 1771258, 'HR00':'HR23'].transpose().quantile(q=[0.05, 0.95])
# ??? the HR05Perc and HR95Perc are not right in the database... am I missing smth?

In [116]:
def fill_hrxxperc(db, subjects):
    for subject in subjects:
        percentiles = np.round(db.loc[db['SubjectNr'] == subject, 'HR00':'HR23'].transpose().quantile(q=[0.05, 0.95]).transpose().rename(columns={0.05:'HR05Perc', 0.95:'HR95Perc'}), 0)
        db.loc[db['SubjectNr'] == subject, ['HR05Perc', 'HR95Perc']] = percentiles

fill_hrxxperc(healthy_df, healthy_subject_nrs)
fill_hrxxperc(asthma_df, asthma_subject_nrs)

### AVGHR_daily
If we guess the missing HRs, we can count it (we have NaN here, if ALL the HRs are missing)

In [117]:
def fill_avghr(db, subjects):
    for subject in subjects:
        avghr = np.round(db.loc[db['SubjectNr'] == subject, 'HR00':'HR23'].transpose().describe().loc['mean'], 0).rename('AVGHR_daily')
        db.loc[db['SubjectNr'] == subject, 'AVGHR_daily'] = avghr

fill_avghr(healthy_df, healthy_subject_nrs)
fill_avghr(asthma_df, asthma_subject_nrs)

In [118]:
asthma_df['AVGHR_daily'].iloc[83]

83.0

### sleeptime & waketime
NA: no sleep data was registered. 579 times. Maybe the time the kid stopped wearing the watch.

What if the subject went to sleep after midnight?

In [ ]:
# UNDER CONSTRUCTION !!!

for i in range(5):# range(len(asthma_df)):
    for stepsXX in asthma_df.columns.values[9:33]:
        if asthma_df.iloc[i][stepsXX] == 0:
            continue
        else:
            asthma_df.loc[i, 'waketime'] = stepsXX
            break

# UNDER CONSTRUCTION !!!

### HEART_PULSE_BPM
I don't think we can fill this, bc "NA: measurement was not performed on this day."

### HRMinSleep and HRMaxSleep
These are quite randomly missing (I dunno how are these calculated), but we could fill it I think.
Comment: (Minimum/Max heart rate measured during sleep) --> between 11pm to 8am maybe? or perhaps better to calculate hrmin and hrmax based on the waketime and sleeptime variables!
Ákos's comment: Ahh true, I was tired. If we decide the "sleeping hours" at the first point, we can do the same here and do what you suggested.


### AVGHR_sleep and AVGHR_wake
How do we know the kid is sleeping or not?
Comment: waketime and sleeptime (yes it has missing data, maybe predict the time they fell asleep based on biometrics such as heartrate? if heartrate down --> fall asleep?
Ákos's comment: Good idea!

### PedsQL_score_baseline
No idea. It's the PedsQL 4.0 score. "Completed at day 0 (screening day).
NA: subject did not complete questionnaire."


### steps_hour_max
If we guess the missing steps, we can count it (we have NaN here, if ALL the steps are missing)
comment: How do you want to guess the missing steps?

### awakeDuration
NA: no sleep data was registered. 579 times. Maybe the time between the kid started and stopped wearing the watch.

### lightSleepDuration
NA: no sleep data was registered. 579 times.

### deepSleepDuration
NA: no sleep data was registered. 579 times.

### wakeUpCount
NA: no sleep data was registered. 579 times.

### fvc_best
NA: measurement was not performed on this day. 3103 times

### fev1_best
NA: measurement was not performed on this day. 3103 times

### pef_best
NA: measurement was not performed on this day. 3103 times

### fev1_ratio_best
NA: measurement was not performed on this day. 3836(not 3103!) times

### fev1_percentage
NA: measurement was not performed on this day. 3103 times

### fvc1_percentage
NA: measurement was not performed on this day. 3103 times

### fev1_ratio_percentage
NA: measurement was not performed on this day. 3103 times

In [ ]:
healthy_df.head(5)

,Index,SubjectNr,DayNo,weekday,dayType,Age,sex,weight,height,BMI_SDS,ethnicity,school_year,sportsyesno,urbanisation,PedsQL_score_baseline,stepsTotalDaily,steps_hour_max,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HRMinSleep,HRMaxSleep,AVGHR_daily,AVGHR_sleep,AVGHR_wake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,wear05H,wear16H,wear24H,HEART_PULSE_BPM,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,sleeptime,waketime,fvc_best,fev1_best,pef_best,grade_fev1,grade_fvc,fev1_ratio_best,fev1_percentage,fvc1_percentage,fev1_ratio_percentage,school_yes_no,screentime,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX
0,1,2853189,0,Thu,holiday,2,Female,12.0,92.0,-1.8,Other / Mixed,Day care,No,Extremely urbanised,97.826087,2806.0,1992.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,480.0,1992.0,156.0,51.0,61.0,39.0,NaN,20.0,56.0,123.0,NaN,NaN,96.0,NaN,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.5,106.5,109.000000,NaN,56.0,NaN,92.0,0,44,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.6,8,4,5.2,3.4,7.5,2.0,25,1.9,9,2
1,2,2853189,1,Fri,holiday,2,Female,12.0,92.0,-1.8,Other / Mixed,Day care,No,Extremely urbanised,97.826087,2564.0,1205.0,151.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,9.0,58.0,NaN,NaN,NaN,1205.0,599.0,NaN,NaN,NaN,52.0,163.0,NaN,NaN,126.0,119.0,128.0,119.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.500000,113.6,145.0,NaN,NaN,20,38,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neither,NaN,7.9,9,4,5.9,4.4,7.2,0.0,0,0.0,-1,-1
2,3,2853189,2,Sat,holiday,2,Female,12.0,92.0,-1.8,Other / Mixed,Day care,No,Extremely urbanised,97.826087,521.0,369.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,369.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN,74.0,NaN,NaN,NaN,NaN,71.0,137.0,NaN,NaN,103.0,NaN,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,NaN,NaN,NaN,NaN,NaN,NaN,102.5,100.333333,NaN,NaN,NaN,NaN,0,25,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neither,NaN,9.2,11,8,7.1,6.0,7.8,0.0,0,0.2,1,1
3,4,2853189,3,Sun,holiday,2,Female,12.0,92.0,-1.8,Other / Mixed,Day care,No,Extremely urbanised,97.826087,250.0,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.0,106.0,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,151.0,156.0,NaN,NaN,154.0,NaN,154.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,19,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3,8,1,6.4,4.5,7.0,0.0,0,0.0,-1,-1
4,5,2853189,4,Mon,school,2,Female,12.0,92.0,-1.8,Other / Mixed,Day care,No,Extremely urbanised,97.826087,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,6,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neither,NaN,7.3,14,1,8.0,4.7,9.8,0.0,0,0.6,3,1


In [ ]:
asthma_df.head(5)

,Unnamed: 0,SubjectNr,DayNo,weekday,dayType,stepsTotalDaily,stepsTotalDetailed,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HrAvgWake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,HRAvgSleep,HRMinSleep,HRMaxSleep,activity_score_parent,school_yes_no,activity_score_child,screentime,sleep_score_child,sleeptime,waketime,fvc_best,fev1_best,pef_best,grade_fev1,grade_fvc,Technique,fev1_ratio_best,fev1_percentage,fvc1_percentage,fev1_ratio_percentage,ACD1,ACD2,ACD3,ACD4,ACD5,ACD6,ACD6_use,ACD6score,BASELINE_ACQ6_score,EOS_ACQ6_score,BASELINE_physical_tot,EOS_physical_tot,BASELINE_emotional_tot,EOS_emotional_tot,BASELINE_school_tot,EOS_school_tot,BASELINE_bodyimage_tot,EOS_bodyimage_tot,BASELINE_treatmentburden_tot,EOS_treatmentburden_tot,BASELINE_health_tot,EOS_health_tot,BASELINE_weight_tot,EOS_weight_tot,BASELINE_respiratory_tot,EOS_respiratory_tot,BASELINE_digestion_tot,EOS_digestion_tot,BASELINE_PAQLQ_total,EOS_PAQLQ_total,BASELINE_PAQLQ_activity,EOS_PAQLQ_activity,BASELINE_PAQLQ_symptoms,EOS_PAQLQ_symptoms,BASELINE_PAQLQ_emotion,EOS_PAQLQ_emotion,PedsQL_score_baseline,EOS_PedsQL_score,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX,pollenTotal,NO,NO2,Nox,O3,Ox,PM10,PM25,Symptomscore,Admission_LY,Age,AgeDiagnosis,AllergicRhin,AtopicAsthma,BirthMonth,BirthYear,BMI,BMI_SDS,CF_Asthma,DailyActiv,DayCare,Dis_Chronic,Eosinophils,ExerciseSymp,Family_his,sex,height,Hosp_treatme,Hospital_Trt,ICS,LABA,Leukotri_Mod,Monoclonals,Oral_Steroid,OtherClass,PancrInsuffi,PetsHome,Pseudomonas,SABA,SchoolYear,Serum_IgE,SmokeHome,sportsyesno,Vaccination,weight,school_year,Urbanisation,urbanisation,totalSleepDuration,wear00,wear01,wear02,wear03,wear04,wear05,wear06,wear07,wear08,wear09,wear10,wear11,wear12,wear13,wear14,wear15,wear16,wear17,wear18,wear19,wear20,wear21,wear22,wear23,wear24H,wear16H,steps15_19,week,steps_hour_max,AVGHR_daily,AVGHR_wake,AVGHR_sleep,CONDITION,EventDay,wear05H
0,1,1771258,0,Tue,school,3723.0,3723.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,137.0,NaN,NaN,605.0,1640.0,NaN,82.0,451.0,245.0,135.0,160.0,164.0,41.0,NaN,68.0,120.0,8.689362e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN,NaN,NaN,NaN,89.8,83.4,87.8,91.5,93.30,77.1,78.6,660.0,16800.0,16440.0,1.0,75.0,61.0,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3232.0,2381.0,4334.00,D,B,3.0,0.736696,0.726555,0.877857,0.822316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,2.666667e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.652174e+09,5.304348e+09,6.4,5.6,6.5,4.2,7.0,6.5,847826087.0,8.586957e+09,6.3,8,4,14.0,85,19.7,5.4,58,0.0,0,0,6.0,32.31,48.88,98.41,3.86,27.83,23.42,13.04,NaN,1,15,7.0,No,Yes,12,2002.0,19.5,-0.1,Controlled asthma,School,0.0,0.0,NaN,No,No,Female,163.2,0.0,0.0,Yes,Yes,No,No,No,No,NaN,Yes,NaN,Yes,4.0,1465.0,No,Yes,0.0,51.9,Secondary school year 4,1.0,Extremely urbanised,33240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,54.0,69.0,NaN,1.0,1640.0,90.0,94.0,NaN,Controlled asthma,NaN,0.0
1,2,1771258,1,Wed,school,10015.0,10015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,607.0,580.0,325.0,180.0,322.0,181.0,491.0,161.0,10.0,744.0,1146.0,4355.0,722.0,142.0,49.0,NaN,82.0,168.0,1.326242e+09,73.6,75.2,70.75,92.0,76.0,70.3,82.3,78.0,99.0,113.0,129.8,95.1,91.60,NaN,108.5,85.25,93.4,95.8,101.8,142.3,107.8,104.20,91.8,93.0,180.0,22200.0,11760.0,0.0,84.0,67.0,121.0,8.0,Ja,8.0,0.5-1 hours,9.0,15.0,13.0,3261.0,2257.0,4509.00,A,A,3.0,0.692119,0.688716,0.885734,0.772559,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.500000,0.166667,2.666667e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.652174e+09,5.304348e+09,6.4,5.6,6.5,4.2,7.0,6.5,847826087.0,8.586957e+09,10.0,12,8,11.3,100,13.1,0.2,2,0.9,2,1,3.0,3.47,32.18,37.4

# One-hot encoding

Unifrom the values within the two datasets in order to merge them later.

In [ ]:
# Looking at the possible values of different columns in the healthy and sick database.
print("Let's allign these values: ")
print(healthy_df['weekday'].unique())
print(asthma_df['weekday'].unique())
print()
print(healthy_df['dayType'].unique())
print(asthma_df['dayType'].unique())
print()
print(healthy_df['school_yes_no'].unique())
print(asthma_df['school_yes_no'].unique())
print()
print(healthy_df['sex'].unique())
print(asthma_df['sex'].unique())
print()
print(healthy_df['school_year'].unique())
print(asthma_df['school_year'].unique())
print()
print(healthy_df['sportsyesno'].unique())
print(asthma_df['sportsyesno'].unique())
print()
print(healthy_df['urbanisation'].unique())
print(asthma_df['urbanisation'].unique())
print()
print(healthy_df['grade_fev1'].unique())
print(asthma_df['grade_fev1'].unique())
print()
print(healthy_df['grade_fvc'].unique())
print(asthma_df['grade_fvc'].unique())
print()
print(healthy_df['fev1_ratio_best'].unique())
print(asthma_df['fev1_ratio_best'].unique())
print("I am not sure why there are only true or nans here???")
print()
print(healthy_df['screentime'].unique())
print(asthma_df['screentime'].unique()) 

Let's allign these values: 
['Thu' 'Fri' 'Sat' 'Sun' 'Mon' 'Tue' 'Wed']
['Tue' 'Wed' 'Thu' 'Fri' 'Sat' 'Sun' 'Mon']

['holiday' 'school' 'weekend']
['school' 'weekend' 'holiday']

[nan 'Neither' 'School' 'Day Care']
[nan 'Ja' 'Nee']

['Female' 'Male']
['Female' 'Male']

['Day care' 'Primary school' 'Secondary school' 'Vocational education']
['Secondary school year 4' 'Primary school year 5'
 'Secondary school year 1' 'Primary school year 7' 'Primary school year 4'
 'Primary school year 8' 'Secondary school year 3' 'Primary school year 3'
 'Secondary school year 2' 'Primary school year 6' nan]

['No' 'Yes']
['Yes' 'No' nan]

['Extremely urbanised' 'Moderately urbanised' 'Very urbanised'
 'Little urbanised']
['Extremely urbanised' 'Not extremely urbanised']

[nan 'E' 'A' 'C' 'U' 'B' 'D' 'F']
['D' 'A' 'B' 'E' 'C' 'U' nan 'F']

[nan 'E' 'A' 'C' 'U' 'B' 'D' 'F']
['B' 'A' 'C' 'E' 'D' 'U' nan 'F']

[nan True]
[0.73669555 0.69211898 0.74384237 ... 0.83669886 0.83624273 0.80671937]
I am not sur

In [ ]:
# school_yes_no
# for healthy: school, daycare or neither
# for asthma: yes or no (school)
# Decision: school and daycare are the same class
# note that nan will be also be a category in the one-hot encoding (unknown)
healthy_df['school_yes_no'] = healthy_df['school_yes_no'].replace({'Neither': 'no', 'Day Care': 'yes', 'School': 'yes'})
asthma_df['school_yes_no'] = asthma_df['school_yes_no'].replace({'Nee': 'no', 'Ja': 'yes'})

# sex
# it doesnt make sense to one hot encode nan for one subject 
# this won't improve classification, we will have to assume a gender 
# Women are more likely to classify as non-binary --> assume female
# https://www.pewresearch.org/social-trends/2022/06/28/americans-complex-views-on-gender-identity-and-transgender-issues/
asthma_df['sex'] = asthma_df['sex'].fillna('Female')

# school year
# I think it will be best to drop this its likely to different in each data set 
healthy_df = healthy_df.drop('school_year', axis=1)
asthma_df = asthma_df.drop('school_year', axis=1)

# sports_yes_no
# A couple subjects didn't fill this in, I think it is a fair assumption to made that 
# if they didn't fill it in than they didn't do sports
asthma_df['sportsyesno'] = asthma_df['sportsyesno'].fillna('No')

# Urbanization
# Decision: Extremely and very will be merged to extremely urbanized
# moderately and little urbanized will be merged to not extremely urbanized
healthy_df['urbanisation'].replace({'Very urbanised': 'Extremely urbanised',
                                    'Moderately urbanised': 'Not extremely urbanised', 
                                    'Little urbanised': 'Not extremely urbanised'}, 
                                   inplace=True)

In [ ]:
print("new classes (they have to be the same): ")
print(healthy_df['weekday'].unique())
print(asthma_df['weekday'].unique())
print()
print(healthy_df['dayType'].unique())
print(asthma_df['dayType'].unique())
print()
print(healthy_df['school_yes_no'].unique())
print(asthma_df['school_yes_no'].unique())
print()
print(healthy_df['sex'].unique())
print(asthma_df['sex'].unique())
print()
print(healthy_df['sportsyesno'].unique())
print(asthma_df['sportsyesno'].unique())
print()
print(healthy_df['urbanisation'].unique())
print(asthma_df['urbanisation'].unique())
print()
print(healthy_df['grade_fev1'].unique())
print(asthma_df['grade_fev1'].unique())
print()
print(healthy_df['grade_fvc'].unique())
print(asthma_df['grade_fvc'].unique())
print()
print(healthy_df['fev1_ratio_best'].unique())
print(asthma_df['fev1_ratio_best'].unique())
print("I am not sure why there are only true or nans here???")
print()
print(healthy_df['screentime'].unique())
print(asthma_df['screentime'].unique()) 

new classes (they have to be the same): 
['Thu' 'Fri' 'Sat' 'Sun' 'Mon' 'Tue' 'Wed']
['Tue' 'Wed' 'Thu' 'Fri' 'Sat' 'Sun' 'Mon']

['holiday' 'school' 'weekend']
['school' 'weekend' 'holiday']

[nan 'no' 'yes']
[nan 'yes' 'no']

['Female' 'Male']
['Female' 'Male']

['No' 'Yes']
['Yes' 'No']

['Extremely urbanised' 'Not extremely urbanised']
['Extremely urbanised' 'Not extremely urbanised']

[nan 'E' 'A' 'C' 'U' 'B' 'D' 'F']
['D' 'A' 'B' 'E' 'C' 'U' nan 'F']

[nan 'E' 'A' 'C' 'U' 'B' 'D' 'F']
['B' 'A' 'C' 'E' 'D' 'U' nan 'F']

[nan True]
[0.73669555 0.69211898 0.74384237 ... 0.83669886 0.83624273 0.80671937]
I am not sure why there are only true or nans here???

[nan '2-4 hours' '0-30 min' '0.5-1 hours' '1-2 hours' '> 4 hours']
[nan '0.5-1 hours' '1-2 hours' '0-30 min' '2-4 hours' '> 4 hours']


In [ ]:
# one-hot encoding
one_hot_asthma = pd.get_dummies(asthma_df[['weekday', 'dayType', 'school_yes_no', 'sex', 'sportsyesno', 'urbanisation',
                                          'grade_fev1', 'grade_fvc', 'screentime']])
one_hot_healthy = pd.get_dummies(healthy_df[['weekday', 'dayType', 'school_yes_no', 'sex', 'sportsyesno', 'urbanisation',
                                          'grade_fev1', 'grade_fvc', 'screentime']])

# Merging
asthma_df = pd.concat([asthma_df, one_hot_asthma], axis=1)
healthy_df = pd.concat([healthy_df, one_hot_healthy], axis=1)

# Make sure to drop old columns (they are replaced with the hot-encoded cols)
healthy_df = healthy_df.drop(columns=['weekday', 'dayType', 'school_yes_no', 'sex', 'sportsyesno', 'urbanisation',
                                          'grade_fev1', 'grade_fvc', 'screentime'])
asthma_df = asthma_df.drop(columns=['weekday', 'dayType', 'school_yes_no', 'sex', 'sportsyesno', 'urbanisation',
                                          'grade_fev1', 'grade_fvc', 'screentime'])

# Result from one-hot encoding

In [ ]:
healthy_df.head(5)

,Index,SubjectNr,DayNo,Age,weight,height,BMI_SDS,ethnicity,PedsQL_score_baseline,stepsTotalDaily,steps_hour_max,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HRMinSleep,HRMaxSleep,AVGHR_daily,AVGHR_sleep,AVGHR_wake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,wear05H,wear16H,wear24H,HEART_PULSE_BPM,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,sleeptime,waketime,fvc_best,fev1_best,pef_best,fev1_ratio_best,fev1_percentage,fvc1_percentage,fev1_ratio_percentage,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX,weekday_Fri,weekday_Mon,weekday_Sat,weekday_Sun,weekday_Thu,weekday_Tue,weekday_Wed,dayType_holiday,dayType_school,dayType_weekend,school_yes_no_no,school_yes_no_yes,sex_Female,sex_Male,sportsyesno_No,sportsyesno_Yes,urbanisation_Extremely urbanised,urbanisation_Not extremely urbanised,grade_fev1_A,grade_fev1_B,grade_fev1_C,grade_fev1_D,grade_fev1_E,grade_fev1_F,grade_fev1_U,grade_fvc_A,grade_fvc_B,grade_fvc_C,grade_fvc_D,grade_fvc_E,grade_fvc_F,grade_fvc_U,screentime_0-30 min,screentime_0.5-1 hours,screentime_1-2 hours,screentime_2-4 hours,screentime_> 4 hours
0,1,2853189,0,2,12.0,92.0,-1.8,Other / Mixed,97.826087,2806.0,1992.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,480.0,1992.0,156.0,51.0,61.0,39.0,NaN,20.0,56.0,123.0,NaN,NaN,96.0,NaN,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.5,106.5,109.000000,NaN,56.0,NaN,92.0,0,44,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.6,8,4,5.2,3.4,7.5,2.0,25,1.9,9,2,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,2853189,1,2,12.0,92.0,-1.8,Other / Mixed,97.826087,2564.0,1205.0,151.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,9.0,58.0,NaN,NaN,NaN,1205.0,599.0,NaN,NaN,NaN,52.0,163.0,NaN,NaN,126.0,119.0,128.0,119.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.500000,113.6,145.0,NaN,NaN,20,38,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.9,9,4,5.9,4.4,7.2,0.0,0,0.0,-1,-1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,2853189,2,2,12.0,92.0,-1.8,Other / Mixed,97.826087,521.0,369.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,369.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN,74.0,NaN,NaN,NaN,NaN,71.0,137.0,NaN,NaN,103.0,NaN,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,NaN,NaN,NaN,NaN,NaN,NaN,102.5,100.333333,NaN,NaN,NaN,NaN,0,25,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.2,11,8,7.1,6.0,7.8,0.0,0,0.2,1,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,2853189,3,2,12.0,92.0,-1.8,Other / Mixed,97.826087,250.0,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.0,106.0,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,151.0,156.0,NaN,NaN,154.0,NaN,154.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,19,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3,8,1,6.4,4.5,7.0,0.0,0,0.0,-1,-1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,2853189,4,2,12.0,92.0,-1.8,Other / Mixed,97.826087,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,6,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.3,14,1,8.0,4.7,9.8,0.0,0,0.6,3,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
asthma_df.head(5)

,Unnamed: 0,SubjectNr,DayNo,stepsTotalDaily,stepsTotalDetailed,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HrAvgWake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,HRAvgSleep,HRMinSleep,HRMaxSleep,activity_score_parent,activity_score_child,sleep_score_child,sleeptime,waketime,fvc_best,fev1_best,pef_best,Technique,fev1_ratio_best,fev1_percentage,fvc1_percentage,fev1_ratio_percentage,ACD1,ACD2,ACD3,ACD4,ACD5,ACD6,ACD6_use,ACD6score,BASELINE_ACQ6_score,EOS_ACQ6_score,BASELINE_physical_tot,EOS_physical_tot,BASELINE_emotional_tot,EOS_emotional_tot,BASELINE_school_tot,EOS_school_tot,BASELINE_bodyimage_tot,EOS_bodyimage_tot,BASELINE_treatmentburden_tot,EOS_treatmentburden_tot,BASELINE_health_tot,EOS_health_tot,BASELINE_weight_tot,EOS_weight_tot,BASELINE_respiratory_tot,EOS_respiratory_tot,BASELINE_digestion_tot,EOS_digestion_tot,BASELINE_PAQLQ_total,EOS_PAQLQ_total,BASELINE_PAQLQ_activity,EOS_PAQLQ_activity,BASELINE_PAQLQ_symptoms,EOS_PAQLQ_symptoms,BASELINE_PAQLQ_emotion,EOS_PAQLQ_emotion,PedsQL_score_baseline,EOS_PedsQL_score,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX,pollenTotal,NO,NO2,Nox,O3,Ox,PM10,PM25,Symptomscore,Admission_LY,Age,AgeDiagnosis,AllergicRhin,AtopicAsthma,BirthMonth,BirthYear,BMI,BMI_SDS,CF_Asthma,DailyActiv,DayCare,Dis_Chronic,Eosinophils,ExerciseSymp,Family_his,height,Hosp_treatme,Hospital_Trt,ICS,LABA,Leukotri_Mod,Monoclonals,Oral_Steroid,OtherClass,PancrInsuffi,PetsHome,Pseudomonas,SABA,SchoolYear,Serum_IgE,SmokeHome,Vaccination,weight,Urbanisation,totalSleepDuration,wear00,wear01,wear02,wear03,wear04,wear05,wear06,wear07,wear08,wear09,wear10,wear11,wear12,wear13,wear14,wear15,wear16,wear17,wear18,wear19,wear20,wear21,wear22,wear23,wear24H,wear16H,steps15_19,week,steps_hour_max,AVGHR_daily,AVGHR_wake,AVGHR_sleep,CONDITION,EventDay,wear05H,weekday_Fri,weekday_Mon,weekday_Sat,weekday_Sun,weekday_Thu,weekday_Tue,weekday_Wed,dayType_holiday,dayType_school,dayType_weekend,school_yes_no_no,school_yes_no_yes,sex_Female,sex_Male,sportsyesno_No,sportsyesno_Yes,urbanisation_Extremely urbanised,urbanisation_Not extremely urbanised,grade_fev1_A,grade_fev1_B,grade_fev1_C,grade_fev1_D,grade_fev1_E,grade_fev1_F,grade_fev1_U,grade_fvc_A,grade_fvc_B,grade_fvc_C,grade_fvc_D,grade_fvc_E,grade_fvc_F,grade_fvc_U,screentime_0-30 min,screentime_0.5-1 hours,screentime_1-2 hours,screentime_2-4 hours,screentime_> 4 hours
0,1,1771258,0,3723.0,3723.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,137.0,NaN,NaN,605.0,1640.0,NaN,82.0,451.0,245.0,135.0,160.0,164.0,41.0,NaN,68.0,120.0,8.689362e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN,NaN,NaN,NaN,89.8,83.4,87.8,91.5,93.30,77.1,78.6,660.0,16800.0,16440.0,1.0,75.0,61.0,93.0,NaN,NaN,NaN,NaN,NaN,3232.0,2381.0,4334.00,3.0,0.736696,0.726555,0.877857,0.822316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,2.666667e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.652174e+09,5.304348e+09,6.4,5.6,6.5,4.2,7.0,6.5,847826087.0,8.586957e+09,6.3,8,4,14.0,85,19.7,5.4,58,0.0,0,0,6.0,32.31,48.88,98.41,3.86,27.83,23.42,13.04,NaN,1,15,7.0,No,Yes,12,2002.0,19.5,-0.1,Controlled asthma,School,0.0,0.0,NaN,No,No,163.2,0.0,0.0,Yes,Yes,No,No,No,No,NaN,Yes,NaN,Yes,4.0,1465.0,No,0.0,51.9,1.0,33240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,54.0,69.0,NaN,1.0,1640.0,90.0,94.0,NaN,Controlled asthma,NaN,0.0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,1771258,1,10015.0,10015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,607.0,580.0,325.0,180.0,322.0,181.0,491.0,161.0,10.0,744.0,1146.0,4355.0,722.0,142.0,49.0,NaN,82.0,168.0,1.326242e+09,73.6,75.2,70.75,92.0,76.0,70.3,82.3,78.0,99.0,113.0,129.8,95.1,91.60,NaN,108.5,85.25,93.

# Delete colums 

The columns deleted are not useful for the prediction, but are for the preprocessing.

In [ ]:
# Just an index we dont need it
healthy_df = healthy_df.drop('Index', axis=1)

# Subjects number are not relevant for predicting
healthy_df = healthy_df.drop('SubjectNr', axis=1)

# We only have ethnicity in healthy_df
healthy_df = healthy_df.drop('ethnicity', axis=1)

In [ ]:
healthy_df.head(5)

,DayNo,Age,weight,height,BMI_SDS,PedsQL_score_baseline,stepsTotalDaily,steps_hour_max,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HRMinSleep,HRMaxSleep,AVGHR_daily,AVGHR_sleep,AVGHR_wake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,wear05H,wear16H,wear24H,HEART_PULSE_BPM,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,sleeptime,waketime,fvc_best,fev1_best,pef_best,fev1_ratio_best,fev1_percentage,fvc1_percentage,fev1_ratio_percentage,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX,weekday_Fri,weekday_Mon,weekday_Sat,weekday_Sun,weekday_Thu,weekday_Tue,weekday_Wed,dayType_holiday,dayType_school,dayType_weekend,school_yes_no_no,school_yes_no_yes,sex_Female,sex_Male,sportsyesno_No,sportsyesno_Yes,urbanisation_Extremely urbanised,urbanisation_Not extremely urbanised,grade_fev1_A,grade_fev1_B,grade_fev1_C,grade_fev1_D,grade_fev1_E,grade_fev1_F,grade_fev1_U,grade_fvc_A,grade_fvc_B,grade_fvc_C,grade_fvc_D,grade_fvc_E,grade_fvc_F,grade_fvc_U,screentime_0-30 min,screentime_0.5-1 hours,screentime_1-2 hours,screentime_2-4 hours,screentime_> 4 hours
0,0,2,12.0,92.0,-1.8,97.826087,2806.0,1992.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,480.0,1992.0,156.0,51.0,61.0,39.0,NaN,20.0,56.0,123.0,NaN,NaN,96.0,NaN,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.5,106.5,109.000000,NaN,56.0,NaN,92.0,0,44,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.6,8,4,5.2,3.4,7.5,2.0,25,1.9,9,2,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,12.0,92.0,-1.8,97.826087,2564.0,1205.0,151.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,9.0,58.0,NaN,NaN,NaN,1205.0,599.0,NaN,NaN,NaN,52.0,163.0,NaN,NaN,126.0,119.0,128.0,119.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.500000,113.6,145.0,NaN,NaN,20,38,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.9,9,4,5.9,4.4,7.2,0.0,0,0.0,-1,-1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,2,12.0,92.0,-1.8,97.826087,521.0,369.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,369.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN,74.0,NaN,NaN,NaN,NaN,71.0,137.0,NaN,NaN,103.0,NaN,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,NaN,NaN,NaN,NaN,NaN,NaN,102.5,100.333333,NaN,NaN,NaN,NaN,0,25,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.2,11,8,7.1,6.0,7.8,0.0,0,0.2,1,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,2,12.0,92.0,-1.8,97.826087,250.0,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.0,106.0,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,151.0,156.0,NaN,NaN,154.0,NaN,154.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,19,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3,8,1,6.4,4.5,7.0,0.0,0,0.0,-1,-1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,2,12.0,92.0,-1.8,97.826087,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,6,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.3,14,1,8.0,4.7,9.8,0.0,0,0.6,3,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
asthma_df.head(5)

,Unnamed: 0,SubjectNr,DayNo,stepsTotalDaily,stepsTotalDetailed,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HrAvgWake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,HRAvgSleep,HRMinSleep,HRMaxSleep,activity_score_parent,activity_score_child,sleep_score_child,sleeptime,waketime,fvc_best,fev1_best,pef_best,Technique,fev1_ratio_best,fev1_percentage,fvc1_percentage,fev1_ratio_percentage,ACD1,ACD2,ACD3,ACD4,ACD5,ACD6,ACD6_use,ACD6score,BASELINE_ACQ6_score,EOS_ACQ6_score,BASELINE_physical_tot,EOS_physical_tot,BASELINE_emotional_tot,EOS_emotional_tot,BASELINE_school_tot,EOS_school_tot,BASELINE_bodyimage_tot,EOS_bodyimage_tot,BASELINE_treatmentburden_tot,EOS_treatmentburden_tot,BASELINE_health_tot,EOS_health_tot,BASELINE_weight_tot,EOS_weight_tot,BASELINE_respiratory_tot,EOS_respiratory_tot,BASELINE_digestion_tot,EOS_digestion_tot,BASELINE_PAQLQ_total,EOS_PAQLQ_total,BASELINE_PAQLQ_activity,EOS_PAQLQ_activity,BASELINE_PAQLQ_symptoms,EOS_PAQLQ_symptoms,BASELINE_PAQLQ_emotion,EOS_PAQLQ_emotion,PedsQL_score_baseline,EOS_PedsQL_score,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX,pollenTotal,NO,NO2,Nox,O3,Ox,PM10,PM25,Symptomscore,Admission_LY,Age,AgeDiagnosis,AllergicRhin,AtopicAsthma,BirthMonth,BirthYear,BMI,BMI_SDS,CF_Asthma,DailyActiv,DayCare,Dis_Chronic,Eosinophils,ExerciseSymp,Family_his,height,Hosp_treatme,Hospital_Trt,ICS,LABA,Leukotri_Mod,Monoclonals,Oral_Steroid,OtherClass,PancrInsuffi,PetsHome,Pseudomonas,SABA,SchoolYear,Serum_IgE,SmokeHome,Vaccination,weight,Urbanisation,totalSleepDuration,wear00,wear01,wear02,wear03,wear04,wear05,wear06,wear07,wear08,wear09,wear10,wear11,wear12,wear13,wear14,wear15,wear16,wear17,wear18,wear19,wear20,wear21,wear22,wear23,wear24H,wear16H,steps15_19,week,steps_hour_max,AVGHR_daily,AVGHR_wake,AVGHR_sleep,CONDITION,EventDay,wear05H,weekday_Fri,weekday_Mon,weekday_Sat,weekday_Sun,weekday_Thu,weekday_Tue,weekday_Wed,dayType_holiday,dayType_school,dayType_weekend,school_yes_no_no,school_yes_no_yes,sex_Female,sex_Male,sportsyesno_No,sportsyesno_Yes,urbanisation_Extremely urbanised,urbanisation_Not extremely urbanised,grade_fev1_A,grade_fev1_B,grade_fev1_C,grade_fev1_D,grade_fev1_E,grade_fev1_F,grade_fev1_U,grade_fvc_A,grade_fvc_B,grade_fvc_C,grade_fvc_D,grade_fvc_E,grade_fvc_F,grade_fvc_U,screentime_0-30 min,screentime_0.5-1 hours,screentime_1-2 hours,screentime_2-4 hours,screentime_> 4 hours
0,1,1771258,0,3723.0,3723.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,137.0,NaN,NaN,605.0,1640.0,NaN,82.0,451.0,245.0,135.0,160.0,164.0,41.0,NaN,68.0,120.0,8.689362e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN,NaN,NaN,NaN,89.8,83.4,87.8,91.5,93.30,77.1,78.6,660.0,16800.0,16440.0,1.0,75.0,61.0,93.0,NaN,NaN,NaN,NaN,NaN,3232.0,2381.0,4334.00,3.0,0.736696,0.726555,0.877857,0.822316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166667,2.666667e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.652174e+09,5.304348e+09,6.4,5.6,6.5,4.2,7.0,6.5,847826087.0,8.586957e+09,6.3,8,4,14.0,85,19.7,5.4,58,0.0,0,0,6.0,32.31,48.88,98.41,3.86,27.83,23.42,13.04,NaN,1,15,7.0,No,Yes,12,2002.0,19.5,-0.1,Controlled asthma,School,0.0,0.0,NaN,No,No,163.2,0.0,0.0,Yes,Yes,No,No,No,No,NaN,Yes,NaN,Yes,4.0,1465.0,No,0.0,51.9,1.0,33240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,54.0,69.0,NaN,1.0,1640.0,90.0,94.0,NaN,Controlled asthma,NaN,0.0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,1771258,1,10015.0,10015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,607.0,580.0,325.0,180.0,322.0,181.0,491.0,161.0,10.0,744.0,1146.0,4355.0,722.0,142.0,49.0,NaN,82.0,168.0,1.326242e+09,73.6,75.2,70.75,92.0,76.0,70.3,82.3,78.0,99.0,113.0,129.8,95.1,91.60,NaN,108.5,85.25,93.

# Setting the columns equal

In [ ]:
# Drop columns in asthma_df that aren't already in healthy_df
asthma_df = asthma_df.drop(columns=asthma_df.columns.difference(healthy_df.columns))

# Sort columns 
asthma_df = asthma_df.reindex(columns=healthy_df.columns)

# Assert the columns are the same
assert all(healthy_df.columns == asthma_df.columns)

In [ ]:
healthy_df.head()

,DayNo,Age,weight,height,BMI_SDS,PedsQL_score_baseline,stepsTotalDaily,steps_hour_max,steps00,steps01,steps02,steps03,steps04,steps05,steps06,steps07,steps08,steps09,steps10,steps11,steps12,steps13,steps14,steps15,steps16,steps17,steps18,steps19,steps20,steps21,steps22,steps23,HR05Perc,HR95Perc,HRMinSleep,HRMaxSleep,AVGHR_daily,AVGHR_sleep,AVGHR_wake,HR00,HR01,HR02,HR03,HR04,HR05,HR06,HR07,HR08,HR09,HR10,HR11,HR12,HR13,HR14,HR15,HR16,HR17,HR18,HR19,HR20,HR21,HR22,HR23,wear05H,wear16H,wear24H,HEART_PULSE_BPM,awakeDuration,lightSleepDuration,deepSleepDuration,wakeUpCount,sleeptime,waketime,fvc_best,fev1_best,pef_best,fev1_ratio_best,fev1_percentage,fvc1_percentage,fev1_ratio_percentage,FG,FHX,FHN,TG,TN,TX,SQ,SP,DR,RH,RHX,weekday_Fri,weekday_Mon,weekday_Sat,weekday_Sun,weekday_Thu,weekday_Tue,weekday_Wed,dayType_holiday,dayType_school,dayType_weekend,school_yes_no_no,school_yes_no_yes,sex_Female,sex_Male,sportsyesno_No,sportsyesno_Yes,urbanisation_Extremely urbanised,urbanisation_Not extremely urbanised,grade_fev1_A,grade_fev1_B,grade_fev1_C,grade_fev1_D,grade_fev1_E,grade_fev1_F,grade_fev1_U,grade_fvc_A,grade_fvc_B,grade_fvc_C,grade_fvc_D,grade_fvc_E,grade_fvc_F,grade_fvc_U,screentime_0-30 min,screentime_0.5-1 hours,screentime_1-2 hours,screentime_2-4 hours,screentime_> 4 hours
0,0,2,12.0,92.0,-1.8,97.826087,2806.0,1992.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,480.0,1992.0,156.0,51.0,61.0,39.0,NaN,20.0,56.0,123.0,NaN,NaN,96.0,NaN,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,115.5,106.5,109.000000,NaN,56.0,NaN,92.0,0,44,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.6,8,4,5.2,3.4,7.5,2.0,25,1.9,9,2,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,12.0,92.0,-1.8,97.826087,2564.0,1205.0,151.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,9.0,58.0,NaN,NaN,NaN,1205.0,599.0,NaN,NaN,NaN,52.0,163.0,NaN,NaN,126.0,119.0,128.0,119.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.500000,113.6,145.0,NaN,NaN,20,38,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.9,9,4,5.9,4.4,7.2,0.0,0,0.0,-1,-1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,2,12.0,92.0,-1.8,97.826087,521.0,369.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,369.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN,74.0,NaN,NaN,NaN,NaN,71.0,137.0,NaN,NaN,103.0,NaN,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,NaN,NaN,NaN,NaN,NaN,NaN,102.5,100.333333,NaN,NaN,NaN,NaN,0,25,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.2,11,8,7.1,6.0,7.8,0.0,0,0.2,1,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,2,12.0,92.0,-1.8,97.826087,250.0,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.0,106.0,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,151.0,156.0,NaN,NaN,154.0,NaN,154.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,19,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3,8,1,6.4,4.5,7.0,0.0,0,0.0,-1,-1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,2,12.0,92.0,-1.8,97.826087,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,6,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.3,14,1,8.0,4.7,9.8,0.0,0,0.6,3,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# TODO imputation
# Obviously we will have to refine this

# This works for
# steps00-23, 
healthy_df.fillna(0, inplace=True)
asthma_df.fillna(0, inplace=True)

# Modeling

In [ ]:
# TODO use eventday column to classify the excurbations

In [ ]:
# Creat a y column for classification
healthy_df['y'] = 0
asthma_df['y'] = 1

# Create the feature matrix X by concatenating the two dataframes
X = pd.concat([asthma_df, healthy_df], axis=0)

# TODO fev1_ratio_best with true and nans only idk why
# For now remove it
object_columns = X.select_dtypes(include='object').columns
print(object_columns)

X = X.select_dtypes(exclude=['object'])
X_object_cols = X.select_dtypes(include=['object'])
print(X_object_cols.dtypes)

y = X['y']

# Split the data into training and testing sets using a 70-30 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the One-Class SVM model to the training data
model = OneClassSVM().fit(X_train[y_train == 1])

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the performance of the model
accuracy = (y_pred == y_test).mean()
print("Accuracy: {:.2f}".format(accuracy))

Index(['fev1_ratio_best'], dtype='object')
Series([], dtype: object)
Accuracy: 0.22


# Making clean dataset for testing

In [ ]:
# Dropping all the columns with a lot of NaNs
asthma_df_first_db = asthma_df.drop(asthma_df.loc[:, 'awakeDuration':'EOS_PedsQL_score'], axis=1)
asthma_df_first_db = asthma_df_first_db.drop(asthma_df_first_db.loc[:, 'pollenTotal':'Nox'], axis=1)
asthma_df_first_db = asthma_df_first_db.drop(asthma_df_first_db.loc[:, 'Ox':'Symptomscore'], axis=1)
asthma_df_first_db = asthma_df_first_db.drop(asthma_df_first_db.loc[:, 'AgeDiagnosis':'AtopicAsthma'], axis=1)
asthma_df_first_db = asthma_df_first_db.drop(asthma_df_first_db.loc[:, 'DayCare':'Family_his'], axis=1)
asthma_df_first_db = asthma_df_first_db.drop(asthma_df_first_db.loc[:, 'Hosp_treatme':'Vaccination'], axis=1)
asthma_df_first_db = asthma_df_first_db.drop(asthma_df_first_db.loc[:, 'Urbanisation':'totalSleepDuration'], axis=1)
asthma_df_first_db = asthma_df_first_db.drop(asthma_df_first_db.loc[:, 'steps15_19':'EventDay'], axis=1)
asthma_df_first_db = asthma_df_first_db.drop(['stepsTotalDetailed', 'HrAvgWake', 'SubjectNr'], axis=1)

# Dropping all the rows with NaN
asthma_df_first_db = asthma_df_first_db.dropna()


healthy_df_first_db = healthy_df.drop(['PedsQL_score_baseline', 'steps_hour_max', 'SubjectNr'], axis=1)
healthy_df_first_db = healthy_df_first_db.drop(healthy_df_first_db.loc[:, 'HRMinSleep':'HRMaxSleep'], axis=1)
healthy_df_first_db = healthy_df_first_db.drop(healthy_df_first_db.loc[:, 'AVGHR_sleep':'AVGHR_wake'], axis=1)
healthy_df_first_db = healthy_df_first_db.drop(healthy_df_first_db.loc[:, 'HEART_PULSE_BPM':'screentime'], axis=1)

healthy_df_first_db = healthy_df_first_db.dropna()

In [ ]:
asthma_df_first_db.to_csv('asthma_dataset_0.csv')
healthy_df_first_db.to_csv('healthy_dataset_0.csv')